## Práctica 0: El Hash Dorado

In [1]:
import random as rd
MOD = 1000000

def es_dorado(coord):
    return hash(coord) % MOD == 0


Nota: coord debe ser una tupla de dos enteros


Nos movemos coordenadas de 1 en 1

In [2]:
def paso_aleatorio(x,y):
    dx, dy = rd.choice([(1,0), (-1,0), (0,1), (0,-1)])
    return x + dx, y + dy

Creamos la funcion lupa, para buscar el hash dorado

In [3]:
def lupa(origen):
    x0, y0 = origen
    x,y = x0, y0
    n=0
    while True:
        x,y = paso_aleatorio(x,y)
        n+=1
        if es_dorado((x,y)):
            print(n)
            return(x,y)

Creamos 8 coordenadas separadas

In [4]:
S = 100_000
starts = [(i*S, 0) for i in range(8)]

Ejecutamos 8 lupas de forma secuencial.

In [5]:
import time

resultados = []

t0 = time.perf_counter()

for start in starts:
    p = lupa(start)
    resultados.append(p)

t1 = time.perf_counter()

print("tiempo total:", t1 - t0)
print("resultados:", resultados)


548359
4322079
12579802
1584570
6504092
5539690
3308955
4886969
tiempo total: 25.12207533302717
resultados: [(41, -50), (100458, 1421), (199419, -5251), (300861, 431), (400125, -169), (499505, -685), (601198, -855), (700147, -258)]


8 lupas concurrentes con hilos (Threading)

In [6]:
import threading
import time

def worker(idx, start, out):
    out[idx] = lupa(start)

resultados = [None] * len(starts)
threads = []

t0 = time.perf_counter()

for i, start in enumerate(starts):
    th = threading.Thread(target=worker, args=(i, start, resultados))
    th.start()
    threads.append(th)

for th in threads:
    th.join()

t1 = time.perf_counter()
print("tiempo total (threads):", t1 - t0)
print("resultados:", resultados)


765472
709643
2604170
2741625
3698347
4058900
5397282
5268659
tiempo total (threads): 15.37463887501508
resultados: [(41, -50), (99366, -844), (200190, -1600), (299622, -799), (400125, -169), (501298, 765), (601030, -2256), (700832, -811)]


In [7]:
import threading
import time

def lupa_con_stop(origen, stop_event):
    x0, y0 = origen
    x, y = x0, y0
    n = 0
    while not stop_event.is_set():
        x, y = paso_aleatorio(x, y)
        n += 1
        if es_dorado((x, y)):
            print(n)
            return (x, y)
    return None


In [8]:
def concurso_threads(starts):
    stop = threading.Event()
    ganador = {"start": None, "coord": None}
    lock = threading.Lock()

    def worker(start):
        coord = lupa_con_stop(start, stop)
        if coord is not None:
            with lock:
                if not stop.is_set():
                    ganador["start"] = start
                    ganador["coord"] = coord
                    stop.set()

    threads = []
    t0 = time.perf_counter()

    for s in starts:
        th = threading.Thread(target=worker, args=(s,))
        th.start()
        threads.append(th)

    for th in threads:
        th.join()

    t1 = time.perf_counter()
    return ganador, (t1 - t0)

ganador, tiempo = concurso_threads(starts)
print("ganador:", ganador)
print("tiempo concurso (threads):", tiempo)


96351
ganador: {'start': (700000, 0), 'coord': (700147, -258)}
tiempo concurso (threads): 0.9437267079483718


8 lupas concurrentes con procesos (Multiprocessing)

In [9]:
import multiprocessing as mp
import time

t0 = time.perf_counter()
with mp.Pool(processes=8) as pool:
    resultados_proc = pool.map(lupa, starts)
t1 = time.perf_counter()

print("tiempo total (procesos):", t1 - t0)
print("resultados:", resultados_proc)

Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'lupa' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda

KeyboardInterrupt: 

Concurso con procesos

In [ ]:
import multiprocessing as mp
import time

def lupa_proceso_concurso(origen, stop_event, q):
    x0, y0 = origen
    x, y = x0, y0
    n = 0

    while not stop_event.is_set():
        x, y = paso_aleatorio(x, y)
        n += 1
        if es_dorado((x, y)):
            q.put((origen, (x, y), n))
            stop_event.set()
            return

def concurso_procesos(starts):
    stop = mp.Event()
    q = mp.Queue()
    procs = []

    t0 = time.perf_counter()

    for s in starts:
        p = mp.Process(target=lupa_proceso_concurso, args=(s, stop, q))
        p.start()
        procs.append(p)

    origen, coord, n = q.get()

    for p in procs:
        p.join()

    t1 = time.perf_counter()
    return {"start": origen, "coord": coord, "iter": n}, (t1 - t0)

ganador, tiempo = concurso_procesos(starts)
print("ganador:", ganador)
print("tiempo concurso (procesos):", tiempo)


Pregunta: ¿hay alguna mejora en el rendimiento al ejecutar las 8 lupas de forma concurrente usando hilos?
No se observa una mejora significativa en el rendimiento respecto a la ejecución secuencial. Aunque las lupas se lanzan de forma concurrente mediante hilos, el tiempo total de ejecución es muy similar al del programa secuencial. Esto se debe a que la implementación de referencia de Python utiliza el Global Interpreter Lock (GIL), que impide que varios hilos ejecuten bytecode de Python simultáneamente. Dado que la búsqueda de hashes es una tarea intensiva en CPU, los hilos compiten por el GIL en lugar de ejecutarse en paralelo, lo que explica que las ejecuciones medidas en el notebook no muestren una reducción apreciable del tiempo total.

Pregunta: En el concurso con 8 lupas concurrentes, ¿tarda más o menos en encontrar un hash dorado que usando una sola lupa? ¿por qué?
De media, el concurso tarda menos en encontrar un hash dorado que una única lupa ejecutándose sola. Cada lupa explora una región distinta del espacio de búsqueda y realiza comprobaciones independientes. Aunque los hilos no se ejecutan en paralelo real debido al GIL, el hecho de lanzar varias lupas incrementa la probabilidad de que alguna de ellas encuentre un hash dorado antes. Desde un punto de vista probabilístico, al realizar varios intentos simultáneos se reduce el tiempo esperado hasta el primer éxito, lo que se refleja en las ejecuciones del notebook.

Pregunta: Utilizando concurrencia basada en procesos, ¿es cierto que mejora el rendimiento? ¿cuál es el motivo?
Sí, el rendimiento mejora claramente cuando se utiliza concurrencia basada en procesos. En este caso, cada proceso tiene su propio intérprete de Python y su propio GIL, lo que permite que los cálculos se ejecuten realmente en paralelo aprovechando varios núcleos de la CPU. Las ejecuciones realizadas muestran una reducción notable del tiempo total frente a las versiones secuencial y con hilos, confirmando que multiprocessing es una solución adecuada para este problema intensivo en CPU.

Pregunta: En el concurso con procesos, ¿se te ocurre alguna otra forma de establecer la comunicación entre procesos? ¿sería más eficiente?
Además de las variables compartidas, se podrían utilizar colas (multiprocessing.Queue) o pipes (multiprocessing.Pipe) para comunicar el resultado del proceso ganador al proceso principal. Otra alternativa sería finalizar el programa en cuanto uno de los procesos termine con éxito, utilizando el código de salida del proceso o la detección de procesos finalizados. Estas soluciones pueden ser más eficientes y simples, ya que reducen la necesidad de sincronización explícita y el acceso a memoria compartida.